In [ ]:
# To Change: port='COM#' = the communication port assigned to the arduino
#            save_path = the file path where you want to save the csv file of data
# To Use: Connect your battery to the arduino. Hit run on the document and wait until you 
#         have collected the data you require.
# To End: Click into the terminal and hit 'Ctrl C' on your keyboard


import serial
import time
import csv

# Establish a connection to the Arduino
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=0.1)

# Give the Arduino some time to reset
time.sleep(2)

# Initialize lists to store time and voltage values
time_values = []
voltage_values = []

# Start time tracking
start_time = time.time()

try:
    while True:
        # Read a line from the serial buffer
        line = arduino.readline().decode('utf-8').strip()
        
        if line:
            print(line)  # Display the line for debugging
            
            # Parse the voltage value from the line if it contains "Voltage:"
            if "Voltage:" in line:
                # Extract the voltage value from the string
                voltage_str = line.split("Voltage: ")[1]
                voltage = float(voltage_str)
                
                # Calculate elapsed time
                elapsed_time = time.time() - start_time
                
                # Append the time and voltage to their respective lists
                time_values.append(elapsed_time)
                voltage_values.append(voltage)

                # Optionally, print the stored values
                print(f"Time: {elapsed_time:.2f} s, Voltage: {voltage:.2f} V")
        
        # Small delay to prevent overwhelming the output
        time.sleep(0.1)

except KeyboardInterrupt:
    # When Ctrl+C is pressed, this block will be executed
    print("\nData collection stopped.")
    arduino.close()  # Close the serial connection

    # Save the collected data to a CSV file
    save_path = r'C:\Users\Razzy\ChemE Car amperage reading\Voltage Reading.csv'
    with open(save_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Time (s)", "Voltage (V)"])
        writer.writerows(zip(time_values, voltage_values))
    
    print(f"Data saved to '{save_path}'.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.polynomial.polynomial import Polynomial
from scipy.optimize import fsolve
from sklearn.metrics import r2_score
from scipy.integrate import simpson

resistance = 1.5 #ohms

# Load the CSV file
csv_file = r'C:\Users\Razzy\ChemE Car amperage reading\2 min bat test - 1 conceptual.csv' # Replace with your file path
data = pd.read_csv(csv_file)
# Remove data points where the voltage is 0
filtered_data = data[data.iloc[:, 1] != 0]

# Extract time and voltage columns after filtering
time = filtered_data.iloc[:, 0]
time = time - time[0] #making sure time starts at 0
voltage = filtered_data.iloc[:, 1]
#making sure the only data points being analyzed are from time 0 to 120
mask = (time >= 0) & (time <= 120)
time = time[mask]
voltage = voltage[mask]
# Fit a high-order polynomial (e.g., 10th order)
degree = 2
coefs = np.polyfit(time, voltage, degree)

poly = np.poly1d(coefs)

# Generate polynomial curve
fitted_voltage = poly(time)

# Compute the derivative of the polynomial
poly_derivative = poly.deriv()

# Define a function to find the roots of
def find_time_for_derivative_equals(value):
    return fsolve(lambda x: poly_derivative(x) - value, time.mean())

# Find the time where the derivative equals -1
time_for_derivative_minus_one = find_time_for_derivative_equals(-1)

#find integral of voltage vs time

totEnergy = simpson(voltage**2/resistance,time)
print('Total energy output over 2 minute period is {}'.format(totEnergy))
# Plot the original data
plt.scatter(time, voltage, marker='.',color='blue', label='Original Data')

#plot the fitted curve
plt.plot(time, fitted_voltage, color='red', label=f'{degree}-Order Fit')

#plot exponential fitted curve
r2_poly = r2_score(fitted_voltage,voltage)
print('r2 score for {} degree polynomial is {}'.format(degree,r2_poly))
print('r2 score for exponential is {}'.format(r2_exp))
# Plot the derivative curve

# Highlight the point where derivative equals -1
#plt.scatter(time_for_derivative_minus_one, poly(time_for_derivative_minus_one), color='purple', zorder=5, label='Derivative = -1')

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.title('Time vs Voltage with Polynomial Fit and Derivative')
plt.legend()

# Show the plot
plt.show()